In [2]:
import geopandas as gpd
import pandas as pd

In [4]:
path_in = "/Users/sofia/Documents/Repos/skytruth-30x30/data/data/raw"
path_out = "/Users/sofia/Documents/Repos/skytruth-30x30/data/data/processed"

In [ ]:
eez = gpd.read_file(path_in + "/World_EEZ_v11_20191118/eez_v11.shp")
eez.head(1)

In [ ]:
eez.columns

In [ ]:
eez.crs

In [ ]:
# Check if eez geometries are valid
sum(eez.geometry.is_valid)

In [ ]:
eez = eez.to_crs('ESRI:54009')
eez['AREA_KM2']= eez.geometry.area/ 1000000
eez.head(5)

In [ ]:
eez.to_file(path_out + "/administrative/eez_area_mollweide.shp", driver="ESRI Shapefile")

In [ ]:
# Reproject to 4326
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_area_mollweide.shp -proj EPSG:4326 -o force /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_area_mollweide.shp

In [ ]:
# List of dictionaries for data in Region_ISO3_PP.txt (list of regions used in the Protected Planet database)
regions_data = [
    {
        'region_iso': 'AS',
        'region_name': 'Asia & Pacific',
        'country_iso_3s': [
            "AFG", "ASM", "AUS", "BGD", "BRN", "BTN", "CCK", "CHN", "COK", "CXR", "FJI", "FSM", "GUM", "HKG", "IDN",
            "IND", "IOT", "IRN", "JPN", "KHM", "KIR", "KOR", "LAO", "LKA", "MAC", "MDV", "MHL", "MMR", "MNG", "MNP",
            "MYS", "NCL", "NFK", "NIU", "NPL", "NRU", "NZL", "PAK", "PCN", "PHL", "PLW", "PNG", "PRK", "PYF", "SGP",
            "SLB", "THA", "TKL", "TLS", "TON", "TUV", "TWN", "VNM", "VUT", "WLF", "WSM"
        ]
    },
    {
        'region_iso': 'AF',
        'region_name': 'Africa',
        'country_iso_3s': [
            "AGO", "BDI", "BEN", "BFA", "BWA", "CAF", "CIV", "CMR", "COD", "COG", "COM", "CPV", "DJI", "DZA", "EGY",
            "ERI", "ESH", "ETH", "GAB", "GHA", "GIN", "GMB", "GNB", "GNQ", "KEN", "LBR", "LBY", "LSO", "MAR", "MDG",
            "MLI", "MOZ", "MRT", "MUS", "MWI", "MYT", "NAM", "NER", "NGA", "REU", "RWA", "SDN", "SEN", "SHN", "SLE",
            "SOM", "SSD", "STP", "SWZ", "SYC", "TCD", "TGO", "TUN", "TZA", "UGA", "ZAF", "ZMB", "ZWE"
        ]
    },
    {
        'region_iso': 'EU',
        'region_name': 'Europe',
        'country_iso_3s': [
            "ALA", "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BGR", "BIH", "BLR", "CHE", "CYP", "CZE", "DEU", "DNK",
            "ESP", "EST", "FIN", "FRA", "FRO", "GBR", "GEO", "GGY", "GIB", "GRC", "HRV", "HUN", "IMN", "IRL", "ISL",
            "ISR", "ITA", "JEY", "KAZ", "KGZ", "LIE", "LTU", "LUX", "LVA", "MCO", "MDA", "MKD", "MLT", "MNE", "NLD",
            "NOR", "POL", "PRT", "ROU", "RUS", "SJM", "SMR", "SRB", "SVK", "SVN", "SWE", "TJK", "TKM", "TUR", "UKR",
            "UZB", "VAT"
        ]
    },
    {
        'region_iso': 'SA',
        'region_name': 'Latin America & Caribbean',
        'country_iso_3s': [
            "ABW", "AIA", "ARG", "ATG", "BES", "BHS", "BLM", "BLZ", "BMU", "BOL", "BRA", "BRB", "CHL", "COL", "CRI",
            "CUB", "CUW", "CYM", "DMA", "DOM", "ECU", "FLK", "GLP", "GRD", "GTM", "GUF", "GUY", "HND", "HTI", "JAM",
            "KNA", "LCA", "MAF", "MEX", "MSR", "MTQ", "NIC", "PAN", "PER", "PRI", "PRY", "SLV", "SUR", "SXM", "TCA",
            "TTO", "UMI", "URY", "VCT", "VEN", "VGB", "VIR"
        ]
    },
    {
        'region_iso': 'PO',
        'region_name': 'Polar',
        'country_iso_3s': [
            "ATF", "BVT", "GRL", "HMD", "SGS"
        ]
    },
    {
        'region_iso': 'NA',
        'region_name': 'North America',
        'country_iso_3s': [
            "CAN", "SPM", "USA"
        ]
    },
    {
        'region_iso': 'GL',
        'region_name': 'Global',
        'country_iso_3s': []
    },
    {
        'region_iso': 'WA',
        'region_name': 'West Asia',
        'country_iso_3s': [
            "ARE", "BHR", "IRQ", "JOR", "KWT", "LBN", "OMN", "PSE", "QAT", "SAU", "SYR", "YEM"
        ]
    }
]

# Convert the region data to a dictionary that maps each country to its region name
country_to_region = {}
for region in regions_data:
    for country in region['country_iso_3s']:
        country_to_region[country] = region['region_name']

In [ ]:
eez['REGIONS'] = eez['ISO_SOV1'].map(country_to_region)
eez.head(2)

In [ ]:
eez.to_file(path_out + "/administrative/eez_mollweide_regions.shp", driver="ESRI Shapefile")

In [ ]:
eez['REGIONS'].unique()

In [ ]:
# Dissolve by relevant fields: REGIONS
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_mollweide_regions.shp -dissolve2 fields=REGIONS -explode -o /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shp

In [ ]:
regions = gpd.read_file(path_out + "/administrative/eez_regions.shp", driver="ESRI Shapefile")
regions.head(2)

In [ ]:
regions.crs

In [ ]:
regions['REGIONS'].unique()

In [ ]:
regions['REGIONS'][regions['REGIONS'].isna()] = 'Antartica'

In [ ]:
# Calculate area of each region
regions['AREA_KM2']= regions.geometry.area/ 1000000

In [ ]:
regions.to_file(path_out + "/administrative/eez_regions.shp", driver="ESRI Shapefile")

In [ ]:
# Reproject to 4326
!mapshaper-xl 16gb /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shp -proj EPSG:4326 -o force /Users/sofia/Documents/Repos/skytruth_30x30/data/processed/administrative/eez_regions.shp

In [ ]:
regions['REGIONS'].unique()

### Extract marine areas

In [38]:
eez = gpd.read_file(path_out + "/administrative/eez_area_mollweide.shp", driver="ESRI Shapefile")
regions = gpd.read_file(path_out + "/administrative/eez_regions.shp", driver="ESRI Shapefile")
hs = gpd.read_file(path_in + "/high_seas/high_seas.shp", driver="ESRI Shapefile")

In [39]:
# Create new column "iso" that has the field "ISO_SOV1" for all rows except those in which ISO_SOV2 and ISO_SOV3 are not null. In such cases concatenate ISO_SOV1, ISO_SOV2 and ISO_SOV3
eez['iso'] = eez['ISO_SOV1']
eez.loc[eez['ISO_SOV2'].notnull(), 'iso'] = eez['ISO_SOV1'] + ";" + eez['ISO_SOV2']
eez.loc[eez['ISO_SOV3'].notnull(), 'iso'] = eez['ISO_SOV1'] + ";" + eez['ISO_SOV2'] + ";" + eez['ISO_SOV3']

In [40]:
len(eez[eez['iso'].str.contains(';')])

49

In [41]:
eez.shape

(281, 33)

In [42]:
# Create a mask for rows with multiple values in 'iso_code'
mask = eez['iso'].str.contains(';', na=False)

# Split the 'iso_code' values and create separate rows only for rows with multiple values
split_rows = eez[mask].copy()
split_rows['iso'] = split_rows['iso'].str.split(';')
split_rows = split_rows.explode('iso')

# Keep rows with single values in 'iso_code'
single_value_rows = eez[~mask]

# Concatenate the exploded rows with the single value rows
eez_new = pd.concat([single_value_rows, split_rows], ignore_index=True)

eez_new.shape

(337, 33)

In [44]:
len(eez_new[eez_new['iso'].str.contains(';')])

0

In [45]:
iso_country_mapping = {
    'USA': 'United States',
    'GBR': 'United Kingdom',
    'NZL': 'New Zealand',
    'FRA': 'France',
    'WSM': 'Samoa',
    'TON': 'Tonga',
    'CHL': 'Chile',
    'URY': 'Uruguay',
    'PER': 'Peru',
    'BRA': 'Brazil',
    'KIR': 'Kiribati',
    'ARG': 'Argentina',
    'AUS': 'Australia',
    'COM': 'Comoros',
    'MDG': 'Madagascar',
    'ZAF': 'South Africa',
    'MUS': 'Mauritius',
    'VUT': 'Vanuatu',
    'NAM': 'Namibia',
    'TLS': 'Timor-Leste',
    'COG': 'Republic of the Congo',
    'AGO': 'Angola',
    'MOZ': 'Mozambique',
    'KEN': 'Kenya',
    'PNG': 'Papua New Guinea',
    'TZA': 'Tanzania',
    'SLB': 'Solomon Islands',
    'SYC': 'Seychelles',
    'COD': 'Democratic Republic of the Congo',
    'ATG': 'Antigua and Barbuda',
    'NLD': 'Netherlands',
    'PRT': 'Portugal',
    'BHS': 'The Bahamas',
    'BRB': 'Barbados',
    'MEX': 'Mexico',
    'CPV': 'Cape Verde',
    'ESP': 'Spain',
    'PAN': 'Panama',
    'CRI': 'Costa Rica',
    'DMA': 'Dominica',
    'DOM': 'Dominican Republic',
    'GTM': 'Guatemala',
    'DNK': 'Denmark',
    'GMB': 'Gambia',
    'GIB': 'Gibraltar',
    'GRD': 'Grenada',
    'SLE': 'Sierra Leone',
    'ISL': 'Iceland',
    'JAM': 'Jamaica',
    'MRT': 'Mauritania',
    'HTI': 'Haiti',
    'KNA': 'Saint Kitts and Nevis',
    'LCA': 'Saint Lucia',
    'VCT': 'Saint Vincent and the Grenadines',
    'TTO': 'Trinidad and Tobago',
    'SLV': 'El Salvador',
    'BLZ': 'Belize',
    'CUB': 'Cuba',
    'SEN': 'Senegal',
    'VEN': 'Venezuela',
    'CAN': 'Canada',
    'NIC': 'Nicaragua',
    'GUY': 'Guyana',
    'COL': 'Colombia',
    'IRL': 'Ireland',
    'GNB': 'Guinea-Bissau',
    'GIN': 'Guinea',
    'CIV': 'Ivory Coast',
    'LBR': 'Liberia',
    'HND': 'Honduras',
    'ECU': 'Ecuador',
    'ESH': 'Western Sahara',
    'SUR': 'Suriname',
    'MAR': 'Morocco',
    'ARE': 'United Arab Emirates',
    'CYP': 'Cyprus',
    'ERI': 'Eritrea',
    'EGY': 'Egypt',
    'GEO': 'Georgia',
    'IRN': 'Iran',
    'LBN': 'Lebanon',
    'LBY': 'Libya',
    'MLT': 'Malta',
    'OMN': 'Oman',
    'SAU': 'Saudi Arabia',
    'LKA': 'Sri Lanka',
    'SDN': 'Sudan',
    'SYR': 'Syria',
    'TGO': 'Togo',
    'GRC': 'Greece',
    'TUR': 'Turkey',
    'MCO': 'Monaco',
    'TUN': 'Tunisia',
    'MNE': 'Montenegro',
    'ALB': 'Albania',
    'BGR': 'Bulgaria',
    'PSE': 'Palestine',
    'KWT': 'Kuwait',
    'IRQ': 'Iraq',
    'BHR': 'Bahrain',
    'QAT': 'Qatar',
    'YEM': 'Yemen',
    'ISR': 'Israel',
    'JOR': 'Jordan',
    'DJI': 'Djibouti',
    'BGD': 'Bangladesh',
    'NGA': 'Nigeria',
    'CMR': 'Cameroon',
    'STP': 'São Tomé and Príncipe',
    'BIH': 'Bosnia and Herzegovina',
    'MHL': 'Marshall Islands',
    'PLW': 'Palau',
    'PHL': 'Philippines',
    'TWN': 'Taiwan',
    'SGP': 'Singapore',
    'THA': 'Thailand',
    'VNM': 'Vietnam',
    'KOR': 'South Korea',
    'BRN': 'Brunei',
    'PRK': 'North Korea',
    'KHM': 'Cambodia',
    'CHN': 'China',
    'EST': 'Estonia',
    'FIN': 'Finland',
    'SWE': 'Sweden',
    'LTU': 'Lithuania',
    'NOR': 'Norway',
    'BEL': 'Belgium',
    'DEU': 'Germany',
    'LVA': 'Latvia',
    'HRV': 'Croatia',
    'ITA': 'Italy',
    'UKR': 'Ukraine',
    'ROU': 'Romania',
    'JPN': 'Japan',
    'IND': 'India',
    'PAK': 'Pakistan',
    'TKM': 'Turkmenistan',
    'AZE': 'Azerbaijan',
    'KAZ': 'Kazakhstan',
    'MMR': 'Myanmar',
    'POL': 'Poland',
    'BEN': 'Benin',
    'SVN': 'Slovenia',
    'MYS': 'Malaysia',
    'ATA': 'Antarctica',
    'TUV': 'Tuvalu',
    'FJI': 'Fiji',
    'FSM': 'Micronesia',
    'GNQ': 'Equatorial Guinea',
    'MDV': 'Maldives',
    'SOM': 'Somalia',
    'NRU': 'Nauru',
    'GAB': 'Gabon',
    'IDN': 'Indonesia',
    'DZA': 'Algeria',
    'GHA': 'Ghana',
    'RUS': 'Russia'
}

In [46]:
def get_name(country):
    return iso_country_mapping.get(country, None)

# Apply the function to create the 'PARENT_ISO' column
eez_new['name_iso'] = eez_new['iso'].apply(get_name)

In [47]:
eez_new.head(2)

,MRGID,GEONAME,MRGID_TER1,POL_TYPE,MRGID_SOV1,TERRITORY1,ISO_TER1,SOVEREIGN1,MRGID_TER2,MRGID_SOV2,...,ISO_SOV3,UN_SOV1,UN_SOV2,UN_SOV3,UN_TER1,UN_TER2,UN_TER3,geometry,iso,name_iso
0,8444.0,American Samoa Exclusive Economic Zone,8670.0,200NM,2204.0,American Samoa,ASM,United States,0.0,0.0,...,NaN,840,NaN,NaN,16.0,NaN,NaN,"POLYGON ((-166.64112 -17.55527, -166.64194 -17...",USA,United States
1,8379.0,Ascension Exclusive Economic Zone,8620.0,200NM,2208.0,Ascension,SHN,United Kingdom,0.0,0.0,...,NaN,826,NaN,NaN,654.0,NaN,NaN,"POLYGON ((-10.93328 -7.88745, -10.93324 -7.889...",GBR,United Kingdom


In [53]:
# List of dictionaries for data in Region_ISO3_PP.txt (list of regions used in the Protected Planet database)
regions_data = [
    {
        'region_iso': 'AS',
        'region_name': 'Asia & Pacific',
        'country_iso_3s': [
            "AFG", "ASM", "AUS", "BGD", "BRN", "BTN", "CCK", "CHN", "COK", "CXR", "FJI", "FSM", "GUM", "HKG", "IDN",
            "IND", "IOT", "IRN", "JPN", "KHM", "KIR", "KOR", "LAO", "LKA", "MAC", "MDV", "MHL", "MMR", "MNG", "MNP",
            "MYS", "NCL", "NFK", "NIU", "NPL", "NRU", "NZL", "PAK", "PCN", "PHL", "PLW", "PNG", "PRK", "PYF", "SGP",
            "SLB", "THA", "TKL", "TLS", "TON", "TUV", "TWN", "VNM", "VUT", "WLF", "WSM"
        ]
    },
    {
        'region_iso': 'AF',
        'region_name': 'Africa',
        'country_iso_3s': [
            "AGO", "BDI", "BEN", "BFA", "BWA", "CAF", "CIV", "CMR", "COD", "COG", "COM", "CPV", "DJI", "DZA", "EGY",
            "ERI", "ESH", "ETH", "GAB", "GHA", "GIN", "GMB", "GNB", "GNQ", "KEN", "LBR", "LBY", "LSO", "MAR", "MDG",
            "MLI", "MOZ", "MRT", "MUS", "MWI", "MYT", "NAM", "NER", "NGA", "REU", "RWA", "SDN", "SEN", "SHN", "SLE",
            "SOM", "SSD", "STP", "SWZ", "SYC", "TCD", "TGO", "TUN", "TZA", "UGA", "ZAF", "ZMB", "ZWE"
        ]
    },
    {
        'region_iso': 'EU',
        'region_name': 'Europe',
        'country_iso_3s': [
            "ALA", "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BGR", "BIH", "BLR", "CHE", "CYP", "CZE", "DEU", "DNK",
            "ESP", "EST", "FIN", "FRA", "FRO", "GBR", "GEO", "GGY", "GIB", "GRC", "HRV", "HUN", "IMN", "IRL", "ISL",
            "ISR", "ITA", "JEY", "KAZ", "KGZ", "LIE", "LTU", "LUX", "LVA", "MCO", "MDA", "MKD", "MLT", "MNE", "NLD",
            "NOR", "POL", "PRT", "ROU", "RUS", "SJM", "SMR", "SRB", "SVK", "SVN", "SWE", "TJK", "TKM", "TUR", "UKR",
            "UZB", "VAT"
        ]
    },
    {
        'region_iso': 'SA',
        'region_name': 'Latin America & Caribbean',
        'country_iso_3s': [
            "ABW", "AIA", "ARG", "ATG", "BES", "BHS", "BLM", "BLZ", "BMU", "BOL", "BRA", "BRB", "CHL", "COL", "CRI",
            "CUB", "CUW", "CYM", "DMA", "DOM", "ECU", "FLK", "GLP", "GRD", "GTM", "GUF", "GUY", "HND", "HTI", "JAM",
            "KNA", "LCA", "MAF", "MEX", "MSR", "MTQ", "NIC", "PAN", "PER", "PRI", "PRY", "SLV", "SUR", "SXM", "TCA",
            "TTO", "UMI", "URY", "VCT", "VEN", "VGB", "VIR"
        ]
    },
    {
        'region_iso': 'PO',
        'region_name': 'Polar',
        'country_iso_3s': [
            "ATF", "BVT", "GRL", "HMD", "SGS"
        ]
    },
    {
        'region_iso': 'NA',
        'region_name': 'North America',
        'country_iso_3s': [
            "CAN", "SPM", "USA"
        ]
    },
    {
        'region_iso': 'WA',
        'region_name': 'West Asia',
        'country_iso_3s': [
            "ARE", "BHR", "IRQ", "JOR", "KWT", "LBN", "OMN", "PSE", "QAT", "SAU", "SYR", "YEM"
        ]
    },
    {
        'region_iso': 'AT', # this region is not in the Protected Planet database
        'region_name': 'Antartica',
        'country_iso_3s': [
            "ATA"
        ]
    }
]

# Convert the region data to a dictionary that maps each country to its region name
country_to_region = {}
name_to_region = {}
for region in regions_data:
    for country in region['country_iso_3s']:
        country_to_region[country] = region['region_iso']
        name_to_region[country] = region['region_name']

In [49]:
eez_new['region'] = eez_new['iso'].map(country_to_region)
eez_new['region'].unique()

array(['NA', 'EU', 'AS', 'SA', 'AF', 'WA', 'AT'], dtype=object)

In [54]:
eez_new['region_name'] = eez_new['iso'].map(name_to_region)
eez_new['region_name'].unique()

array(['North America', 'Europe', 'Asia & Pacific',
       'Latin America & Caribbean', 'Africa', 'West Asia', 'Antartica'],
      dtype=object)

In [55]:
marine_areas = eez_new.groupby(['iso', 'name_iso']).agg({'AREA_KM2': 'sum'}).reset_index()
marine_areas = marine_areas.rename(columns={'iso': 'location_id', 'name_iso':'location_name', 'AREA_KM2': 'total_marine_area'})
marine_areas['location_type'] = 'country'
marine_areas.head(2)

,location_id,location_name,total_marine_area,location_type
0,AGO,Angola,498908.577009,country
1,ALB,Albania,12177.287755,country


In [57]:
regions_areas = eez_new.groupby(['region', 'region_name']).agg({'AREA_KM2': 'sum'}).reset_index()
regions_areas = regions_areas.rename(columns={'region': 'location_id', 'region_name':'location_name', 'AREA_KM2': 'total_marine_area'})
regions_areas['location_type'] = 'region'
regions_areas.head(2)

,location_id,location_name,total_marine_area,location_type
0,AF,Africa,1.495538e+07,region
1,AS,Asia & Pacific,5.269208e+07,region


In [59]:
global_area = pd.DataFrame({'location_id': ['GLOB'], 'location_name': ['Worldwide'], 'total_marine_area': [361000000], 'location_type': ['worldwide']}) 
hs_area = pd.DataFrame({'location_id': ['ABNJ'], 'location_name': ['High Seas'], 'total_marine_area': [hs['area_km2'].values[0]], 'location_type': ['country']})

In [63]:
# concat gl_df and hs_df to marine_areas
marine_areas2 = pd.concat([marine_areas, regions_areas, global_area, hs_area], ignore_index=True)
marine_areas2


,location_id,location_name,total_marine_area,location_type
0,AGO,Angola,4.989086e+05,country
1,ALB,Albania,1.217729e+04,country
2,ARE,United Arab Emirates,5.821593e+04,country
3,ARG,Argentina,2.897629e+06,country
4,ATA,Antarctica,8.842860e+06,country
...,...,...,...,...
162,NA,North America,1.791826e+07,region
163,SA,Latin America & Caribbean,2.107800e+07,region
164,WA,West Asia,1.456969e+06,region
165,GLOB,Worldwide,3.610000e+08,worldwide


In [64]:
marine_areas2['location_type'].unique()

array(['country', 'region', 'worldwide'], dtype=object)

### Clean WDPA dataset

In [5]:
# Read WDPA data
poly1 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_0/WDPA_WDOECM_Sep2023_Public_marine_shp-polygons.shp")
point1 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_0/WDPA_WDOECM_Sep2023_Public_marine_shp-points.shp")
poly2 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_1/WDPA_WDOECM_Sep2023_Public_marine_shp-polygons.shp")
point2 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_1/WDPA_WDOECM_Sep2023_Public_marine_shp-points.shp")
poly3 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_2/WDPA_WDOECM_Sep2023_Public_marine_shp-polygons.shp")
point3 = gpd.read_file(path_in + "/WDPA_WDOECM_Sep2023_Public_marine_shp/WDPA_WDOECM_Sep2023_Public_marine_shp_2/WDPA_WDOECM_Sep2023_Public_marine_shp-points.shp")
dataframes = [poly1, point1, poly2, point2, poly3, point3]

KeyboardInterrupt: 

**Convert points to polygons and merge all wdpa in one dataset**

In [ ]:
# Calculate radius based on REP_AREA
def calculate_radius(rep_area):
    return (rep_area / 3.14159265358979323846) ** 0.5

# Iterate through the list and process the desired dataframes
for idx in [1, 3, 5]:
    # Get the dataframe at the specified index
    gdf = dataframes[idx]

    # Reproject in Mollweide
    gdf = gdf.to_crs('ESRI:54009')

    # Transform the reported area from square kilometers to square meters
    gdf['REP_AREA_m'] = gdf['REP_AREA'] * 1000000

    # Create the "radius" column by applying the calculate_radius function to the "REP_AREA" column
    gdf['radius'] = gdf['REP_AREA_m'].apply(calculate_radius)

    # Create buffers around the points using the "radius" column
    gdf_buffered = gdf.copy()
    gdf_buffered['geometry'] = gdf.apply(lambda row: row.geometry.buffer(row['radius']), axis=1)

    # Reproject back to WGS84
    gdf_buffered = gdf_buffered.to_crs('EPSG:4326')

    # Remove rows with invalid geometries
    gdf_buffered = gdf_buffered[gdf_buffered['geometry'].is_valid]
    
    # Update the original dataframe with the buffered data
    dataframes[idx] = gdf_buffered

# Merge all dataframes
merged_mpa_all = pd.concat(dataframes)
len(merged_mpa_all)

18613

In [ ]:
# Save the wdpa dataframe as a shapefile
merged_mpa_all.to_file(path_out + "/wdpa/merged_wdpa_all.shp")